In [124]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime

In [78]:
base_path = '/Users/yifanli/Github/fidelity-portfolio-tracker'
os.chdir(base_path)

In [134]:
from support_functions import portfolio, load_transaction, load_position

In [135]:
importlib.reload(load_transaction)
load_transaction = load_transaction.load_transaction
importlib.reload(load_position)
load_position = load_position.load_position

In [136]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
current_date = datetime.strptime('Nov-12-2024', '%b-%d-%Y')

transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)

In [137]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

In [138]:
current_portfolio = Portfolio(transactions=transactions, position=position)

In [139]:
current_portfolio.individual_transactions['Symbol'].unique()

array(['FZFXX', 'NKE', 'SBUX', 'AAPL', 'AXP', 'JPM', 'GOOGL', 'AMZN',
       '912797LL9', 'Transfer', '912797JK4', '912797MB0', 'COKE',
       '912797LC9', '912797HT7', '912797GN1', 'FXAIX', '912797HQ3',
       'TSLA', '912797HG5', '912797GZ4', '912797LR6', '912797KL0', 'MCD',
       '912797LH8', 'MSFT', '912797KV8', '912797KT3', '912797HP5',
       '912797KC0', '912797GW1', '912797LE5', '912797KA4', '912797KB2',
       '912797JU2', '912797JT5', 'FSPSX', 'FSKAX', '912797GB7',
       '912797HS9'], dtype=object)

In [155]:
test_symbol = 'AAPL'
temp_transactions = current_portfolio.individual_transactions
test_transactions = temp_transactions[temp_transactions['Symbol']==test_symbol]
temp_position = current_portfolio.individual_position
test_position = temp_position[temp_position['Symbol']==test_symbol]

In [156]:
dates = test_transactions['Run Date']
cash_flows = test_transactions['Amount ($)'].values
PV = test_position['Current Value'].values

In [160]:
time_diffs = np.array([(datetime.now().date() - date).days for date in dates]) / 365.25
def npv(rate, cash_flows, time_diffs):
    return sum(cash_flows * (1 + rate) ** time_diffs)
for rate in np.arange(-0.999, 5, 0.001):
        if npv(rate, -cash_flows, time_diffs) > PV:
            break
rate

0.19800000000000106

In [ ]:
from datetime import datetime
import numpy as np

def irr(cash_flows, dates, PV):
    # Calculate time differences in years
    time_diffs = np.array([(datetime.now().date() - date).days for date in dates]) / 365.25

    # Define NPV function
    def npv(rate, cash_flows, time_diffs):
        return sum(cash_flows * (1 + rate) ** time_diffs)

    # Iterate over possible rates to find the IRR
    for rate in np.arange(-0.999, 5, 0.001):
        if npv(rate, cash_flows, time_diffs) > PV:
            break

    return rate